# 9.2 Data modeling 

#### This week:
**Miscellaneous:**
* data storage in a pckle

***Data modeling:**
* Standard normal distribution
* Moments of distribution
* Least-square fitting of arbitrary curve

## Miscellaneous
### A useful python data storgage strategy: pickle

In [ ]:
dict = {}
dict['set1'] = array([1,45,16])
dict['mb2'] = {}; dict['mb2'][4]=(1,2); dict['mb2']['Paula']=5
all = ['Orange',(3,4)]

In [ ]:
import pickle

In [ ]:
pickle_out = open("datacontainer.pickle","wb")
pickle.dump([dict,all], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("datacontainer.pickle","rb")
thing1,thing2 = pickle.load(pickle_in)
pickle_in.close()

## Standard normal distribution

The [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) describes random processes, such as repeat measurements of a physical quantity. It plays an important role as a model distribution in many areas.

Roadmap:
1. Create data set with a normal distribution.
2. Plot a histogram of the distribution.
3. Overplot the properly scaled propability density of the normal distribution.


In [ ]:
#%pylab ipympl
%pylab nbagg
random.standard_normal?

In [ ]:
%pylab ipympl

ifig=1;close(ifig);figure(ifig)
a=random.standard_normal((2,300))
plot(a[0],a[1],'o')

Plot the usual histogram of the distribtion. We use both `a` arrays and combine them (_loose the shape_) with the `.flatten` method.

In [ ]:
ifig=2;close(ifig);figure(ifig)
#ifig=2;figure(ifig)
n=10
ah=hist(a.flatten(),n)


In [ ]:
# plot distribution
yh=ah[0]
xh=ah[1][0:-1]+0.5*diff(ah[1])
plot(xh,yh,'o--')

But default `hist` will represent the number of samples in each bin. However, often it is desirable to plot the value of the probability density function at the bin, normalized such that the integral over the range is 1.

Normalization of discretized distribution: $$\zeta_\mathrm{dens} = \frac{(x_\mathrm{max}-x_\mathrm{in}) N}{n}$$ where $N$ is the number of values and $n$ is the number of bars in the histogram. Divide each bin value by $\zeta_\mathrm{dens}$.

The same is accomplished with the `numpy.hist` option `density=True`.

In [ ]:
ifig=22;close(ifig);figure(ifig)
#ifig=2;figure(ifig)
n=10
ad=hist(a.flatten(),n,density=True)


In [ ]:
# plot distribution
yd=ad[0]
xd=ad[1][0:-1]+0.5*diff(ad[1])
plot(xd,yd,'o--')

#### Probability density of the normal distribution
$$ f(x | \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp -\frac{(x-\mu)^2}{2\sigma^2})$$
where $\mu$ is the mean, $\sigma$ is the standard deviation and $\sigma^2$ is the variance.

In [ ]:
def pdf_normal(x,mu,var):
    '''Probaility density function of normal distribution'''
    thing = sqrt(2*pi*var)
    thang = -(x-mu)**2/(2.*var)
    return exp(thang)/thing

In [ ]:
close(3);figure(3)
xmin = ad[1][0]; xmax = ad[1][-1]
print(xmin,xmax)
xx = linspace(xmin,xmax,1000)
plot(xx,pdf_normal(xx,0,1))

In [ ]:
# add to discrete distribution density Figure 22
figure(22)
plot(xx,pdf_normal(xx,0,1))

Once more, make a clean plot of PDF (probability density function) and normalized data:

In [ ]:
ifig=3;close(ifig);figure(ifig)
plot(xd,yd,'o')
plot(xx,pdf_normal(xx,0,1))

## Moments of distribution

**Literature:** Press etal. _Numerical Recipies_, Chapter 14.1

Moments are the sums of integer powers of data values $(x_1, x_2,
x_3, \dots, x_\mathrm{N})$. They characterize the distribution of
data values if the data shows a sufficiently strong tendency to 
cluster around some particular value.

### Mean

$1^{st}$ moment:

$$  <x> = \bar{x} = \frac{1}{N} \sum_{j=1}^N x_j  $$

### Variance/width

$2^{nd}$ moment:

$$ var(x_1, x_2, x_3, \dots, x_N) = \frac{1}{N-1} \sum_{j=1}^N (x_j - \bar{x})^2  $$

and related to that the standard deviation:

$$ \sigma(x_1, x_2, x_3, \dots, x_N) = \sqrt{var(x_1, x_2, x_3, \dots, x_N)} $$


###  Skew

The $3^{rd}$ moment indicates the asymmetry of the distribution in
terms of a tilt:

$$ skew(x_1, x_2, x_3, \dots, x_N) = \frac{1}{N} \sum_{j=1}^N
\left(\frac{x_j - \bar{x}}{\sigma}\right)^3 $$

## Function to calculate mean and std deviation

Write a function that will take an array of data values and calculates the mean and standard deviation. Add appropriate documentation and make sure that common user errors are escaped.

In [ ]:
def my_mean_var(x):
    N = len(x)
    m = sum(x)/N
    var = sum((x-m)**2)/(N-1)
    return m,var

In [ ]:
my_mean_var(a.flatten())

In [ ]:
# of course, libraries can do that as well: numpy
print(mean(a.flatten()),var(a.flatten()))

### A skewed distribution
Let's generate a skewed distribution.

In [ ]:
# skew
y = lambda x: sqrt(1-x**2)
x=linspace(-1,1,500)
close(115);figure(115)
plot(x,y(x),'o')

In [ ]:
close(13);figure(13)
yh = hist(y(x),10)

In [ ]:
from scipy import stats

In [ ]:
stats.skew?

In [ ]:
stats.skew(y(x))

This distribution has
a longer left tail and the mass of the distribution is concentrated on the right of the figure which is associated with a 
[negative skew](https://en.wikipedia.org/wiki/Skewness#/media/File:Negative_and_positive_skew_diagrams_(English).svg).


## Linear correlation

**Literature:** Press etal. _Numerical Recipies_, Chapter 14.5

If we know that two data sets are correlated than the linear correlation coefficient provides a measure of how well they are correlated. This is a different question than _Are the two data sets correlated?_ - and a 
slightly modified approach is needed to answer that question; see discussion Chapter 14.5, Numerical Recipes.

For $N$ pairs of values $(x_i,y_i)$ with $i=1 \dots N$ this coefficient (also called the product-moment correlation coefficient, or _Pearson's r_) is given by

$$ r = \frac{\sum_i (x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_i (x_i-\bar{x})^2}\sqrt{\sum_i (y_i-\bar{y})^2}}$$

An example of a real-world application of the _Pearson's r_ correlation analysis can be found in this [paper](http://adsabs.harvard.edu/abs/2018JPhG...45e5203D).

Let's look at two data sets, one with and one without correlation.

In [ ]:
noise_level=0.4
noisy_y=exp(-x)+noise_level*randn(len(x))
close(19);figure(19)
plot(x,noisy_y,'o')
plot(x,randn(len(x)),'s')

In [ ]:
# strongly correlated set of data pairs
stats.pearsonr(x,2*x)

In [ ]:
# a clearly uncorrelated set of data pairs
stats.pearsonr(x,randn(len(x)))

In [ ]:
# strongly correlated set of data pairs
stats.pearsonr(x,noisy_y)

### Fitting data with a model 

**Literature:** Press etal. _Numerical Recipies_, Chapter 15.1 - 15.2

#### $\chi$-square as merit function
Often we have some experimental data, and we have a physics based
model in the form of a simple equation that we expect to reproduce
trends observed in the data. The model contains undetermined
parameters, maybe representing unresolved sub-grid physics.

Again, we have $N$ pairs of values $(x_i,y_i)$ with $i=1
\dots N$, and we have a model in the form
$$ y(x) = y(x; a_1 \dots a_M)$$

In order to determine the parameters $(a_1 \dots a_M)$ that
provide the _maximum likelyhood_ for the data to be a representation
of the model (specified by the parameter values $a_j$) we need to
adopt a _merit function_ that is arranged so that a minimum in the
merit function yields the _best-fit parameters_.

Such a _merit function_ may be based on the least-squares fit:

mimimize over the parameter values $a_j$:
$$ \sum_{i=1}^N [y_i -  y(x; a_1 \dots a_M) ]^2$$

However, this does not yet include the statistical error on each
$y_i$. Therefore we minimize instead over
$$\chi^2 =  \sum_{i=1}^N \left( \frac{y_i -  y(x; a_1 \dots a_M) }{\sigma_i}\right )^2 $$
where $\sigma_i$ is the standard deviation of the point $(x_i,y_i)$.

#### Fitting a straight line

The most simple model would involve a linear correlation
$$ y(x) = y(x; a, b) = a + bx$$



##### What is $\chi^2$ for this model?
$$
\chi^2(a,b) = \sum_{i=1}^N \left ( \frac{y_i - a - bx_i}{\sigma_i} \right )^2
$$

##### What are the parameters $a$ and $b$?
Derive a condition for $a$ and $b$ considering that $\chi^2$ has a minimum with respect to $a$ and $b$ when $\frac{\partial \chi^2}{\partial a} = 0$ and $\frac{\partial \chi^2}{\partial b} = 0$

With 
$$ S \equiv \sum_{i=1}^N \frac{1}{\sigma_i^2} $$
$$ S_x \equiv \sum_{i=1}^N \frac{x_i}{\sigma_i^2}\mathrm{,\ } S_y \equiv \sum_{i=1}^N \frac{y_i}{\sigma_i^2} $$
$$ S_{xx} \equiv \sum_{i=1}^N \frac{x_i^2}{\sigma_i^2}\mathrm{,\ } S_{xy} \equiv \sum_{i=1}^N \frac{x_i y_i}{\sigma_i^2} $$

we get

$$ aS +bS_x = S_y$$
$$ aS_x + b S_{xx} = S_{xy}$$

with the solution 

$$\Delta  \equiv SS_{xx} - (S_x)^2$$

$$a = \frac{S_{xx}S_y-S_xS_{xy}}{\Delta}$$

$$b = \frac{SS_{xy}S_y-S_xS_{y}}{\Delta}$$


This is how you could implement this yourself.
But this is only the first step. We would also need to determine the _goodness-of-fit_. To make it easy, let's use some libraries.

## Least-square fitting of arbitrary curve

In [ ]:
%pylab ipympl

In [ ]:
x=linspace(0,5,100)

In [ ]:
ifig=5;close(ifig);figure(ifig)
def func_explore(x):
    return exp(-x)
#    return x
plot(x,func_explore(x),'o')

In [ ]:
noise_level=0.2
noisy_y=func_explore(x)+noise_level*randn(len(x))
figure(5)
plot(x,noisy_y,'o')

In [ ]:
ifig=6;close(ifig);figure(ifig)
hist(noisy_y,6,density=True)
#hist(func_explore(x),6)

In [ ]:
from scipy.optimize import curve_fit
#curve_fit?

In [ ]:
# try to fit the data with these two model functions
def model_func(x,a,b,c):
#    return a*x**3+b*x**2+c
    return a*exp(b*x)+c
fitpars, covmat = curve_fit(model_func,x,noisy_y,p0=[1.5,-1.5,0.1])

In [ ]:
variances = covmat.diagonal()
std_devs = np.sqrt(variances)
print(fitpars,std_devs)

In [ ]:
ifig=7;close(ifig);figure(ifig)

plot(x,func_explore(x),label='org function')    
plot(x,noisy_y,'o',label='noised data')   
plot(x,model_func(x,fitpars[0],fitpars[1],fitpars[2]),'--',\
     label='fitted function') # fitted model function
legend(loc=0)

### Alternative libraries

There is another fitting method called `polyfit` in the `numpy` package. It is specifically designed to fit data with a power law. Another option is `numpy.linalg.lstsq`. You will have to try them, read the documentation of what algorithms they use, what limitations are specified and try to assess which methods best serves your needs.